# 1. Introdução

O classificador em cascata Haar é um algoritmo proposto por Viola e Jones (2001), que permite a detecção rápida e precisa de objetos (como rostos) em imagens. Ele se baseia em três pilares principais:

* Características Haar-like
* AdaBoost
* Classificação em cascata

# 2. Estratégia

A imagem é varrida com uma janela deslizante em vários tamanhos (para detectar rostos em diferentes escalas).

Cada janela passa pela cascata.

Se aprovada em todos os estágios, a janela é marcada como “contém um rosto”.

Não-máxima supressão (NMS) pode ser usada para eliminar sobreposições.

In [ ]:
! [ ! -d "/content/pessoas.jpg" ] && gdown -O /content/pessoas.jpg "19Q9M_vCwIxyUK4C45F2ZjRZcKUoBUugq"

! [ ! -d "/content/haarcascade_frontalcatface.xml" ] && gdown -O /content/haarcascade_frontalcatface.xml "1wXZTJRTKCsDGNl_7AbdFh979BMOTtrtv"

! [ ! -d "/content/haarcascade_frontalcatface_extended.xml" ] && gdown -O /content/haarcascade_frontalcatface_extended.xml "1KD1Lv7PqmlagGydzIa5AVrqq0GaParUx"

! [ ! -d "/content/cats_vs_dogs" ] && gdown -O /content/cats_vs_dogs.zip "1VuxwfbLZDXPt_0yKTauh4LRIhqMX1SdR" &&  unzip -q /content/cats_vs_dogs.zip -d /content && rm /content/cats_vs_dogs.zip


# 3. Exemplo Didático

Trabalha com imagens em escala de cinza (matriz NumPy),

Simula uma janela de 24×24 pixels (como no Viola-Jones),

Aplica 3 estágios, cada um com uma "feature" simples.

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

# === Funções Haar-like simplificadas ===
def haar_horizontal_edge_feature(img):
    h, w = img.shape
    return img[h//2:, :].sum() - img[:h//2, :].sum()

def haar_vertical_edge_feature(img):
    h, w = img.shape
    return img[:, w//2:].sum() - img[:, :w//2].sum()

def haar_diagonal_feature(img):
    h, w = img.shape
    mask = np.tri(h, w, dtype=bool)
    return img[~mask].sum() - img[mask].sum()

# === Cascata de detecção com imagens ===
def run_cascade_with_visuals(img, label="Amostra"):
    h, w = img.shape
    visuals = []
    logs = []

    # Estágio 1: borda horizontal
    feat1 = haar_horizontal_edge_feature(img)
    stage1 = cv2.cvtColor(img.copy(), cv2.COLOR_GRAY2BGR)
    cv2.rectangle(stage1, (0, 0), (w-1, h//2), (255, 0, 0), 1)
    cv2.rectangle(stage1, (0, h//2), (w-1, h-1), (0, 255, 0), 1)
    if feat1 < 500:
        logs.append("Estágio 1: FALHOU")
        visuals.append((stage1, f"{label} - Estágio 1: FALHOU"))
        return False, logs, visuals
    logs.append("Estágio 1: OK")
    visuals.append((stage1, f"{label} - Estágio 1: OK"))

    # Estágio 2: borda vertical
    feat2 = haar_vertical_edge_feature(img)
    stage2 = cv2.cvtColor(img.copy(), cv2.COLOR_GRAY2BGR)
    cv2.rectangle(stage2, (0, 0), (w//2, h-1), (255, 0, 0), 1)
    cv2.rectangle(stage2, (w//2, 0), (w-1, h-1), (0, 255, 0), 1)
    if feat2 < 300:
        logs.append("Estágio 2: FALHOU")
        visuals.append((stage2, f"{label} - Estágio 2: FALHOU"))
        return False, logs, visuals
    logs.append("Estágio 2: OK")
    visuals.append((stage2, f"{label} - Estágio 2: OK"))

    # Estágio 3: padrão diagonal
    feat3 = haar_diagonal_feature(img)
    stage3 = cv2.cvtColor(img.copy(), cv2.COLOR_GRAY2BGR)
    for i in range(0, min(h, w), 2):
        cv2.line(stage3, (0, i), (i, 0), (0, 255, 255), 1)
    if feat3 < 200:
        logs.append("Estágio 3: FALHOU")
        visuals.append((stage3, f"{label} - Estágio 3: FALHOU"))
        return False, logs, visuals
    logs.append("Estágio 3: OK")
    visuals.append((stage3, f"{label} - Estágio 3: OK"))

    return True, logs, visuals

# === Gerar imagens de exemplo ===

def generate_fake_face():
    img = np.zeros((24, 24), dtype=np.uint8)
    img[12:, :] = 200
    img[:, 12:] += 50
    np.fill_diagonal(img, 255)
    return img

def generate_negative_sample():
    np.random.seed(42)
    return np.random.randint(0, 100, (24, 24), dtype=np.uint8)

# === Exibir os resultados lado a lado ===

def show_cascade_visuals(pos_visuals, neg_visuals):
    fig, axs = plt.subplots(2, 3, figsize=(12, 6))

    for i in range(3):
        # Positiva
        if i < len(pos_visuals):
            axs[0, i].imshow(pos_visuals[i][0])
            axs[0, i].set_title(pos_visuals[i][1])
        else:
            axs[0, i].imshow(np.zeros((24, 24, 3), dtype=np.uint8))
            axs[0, i].set_title("Estágio não alcançado")
        axs[0, i].axis("off")

        # Negativa
        if i < len(neg_visuals):
            axs[1, i].imshow(neg_visuals[i][0])
            axs[1, i].set_title(neg_visuals[i][1])
        else:
            axs[1, i].imshow(np.zeros((24, 24, 3), dtype=np.uint8))
            axs[1, i].set_title("Estágio não alcançado")
        axs[1, i].axis("off")

    axs[0, 0].set_ylabel("Imagem Positiva", fontsize=12)
    axs[1, 0].set_ylabel("Imagem Negativa", fontsize=12)
    plt.tight_layout()
    plt.show()

# === Rodar tudo ===

if __name__ == "__main__":
    pos_img = generate_fake_face()
    neg_img = generate_negative_sample()

    _, pos_log, pos_visuals = run_cascade_with_visuals(pos_img, label="Positiva")
    _, neg_log, neg_visuals = run_cascade_with_visuals(neg_img, label="Negativa")

    show_cascade_visuals(pos_visuals, neg_visuals)

    print("\n[Log da imagem positiva]:")
    for line in pos_log:
        print(line)

    print("\n[Log da imagem negativa]:")
    for line in neg_log:
        print(line)
